In [18]:
import numpy as np
from scipy.linalg import eigh, qr, null_space
import matplotlib.pyplot as plt
from scipy.sparse import kron, identity, csr_matrix, lil_matrix, dok_matrix, coo_matrix, issparse, diags, eye
from scipy.sparse.linalg import norm as sparse_norm
from scipy.sparse.linalg import eigsh, eigs
from scipy.special import factorial, comb
from scipy.optimize import curve_fit
from qutip import Qobj, ptrace, entropy_vn, qeye, tensor
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from joblib import Parallel, delayed
from itertools import combinations
#from quspin.basis import spin_basis_1d, spin_basis_general
#import tenpy as tp

In [2]:
# spinful jw - single site operators

# Four basis states for each site: |0>, |↑>, |↓>, |↑↓>
ket_empty = csr_matrix([[1], [0], [0], [0]], dtype=np.complex128)  # |0>
ket_up = csr_matrix([[0], [1], [0], [0]], dtype=np.complex128)     # |↑>
ket_down = csr_matrix([[0], [0], [1], [0]], dtype=np.complex128)   # |↓>
ket_both = csr_matrix([[0], [0], [0], [1]], dtype=np.complex128)   # |↑↓>

# Single-site operators for spin-up
# Annihilation operator c_↑
c_up = csr_matrix([
    [0, 1, 0, 0],  # |0> -> 0
    [0, 0, 0, 0],  # |↑> -> |0>
    [0, 0, 0, 1],  # |↓> -> 0
    [0, 0, 0, 0]   # |↑↓> -> |↓>
], dtype=np.complex128)

# Creation operator c†_↑
c_up_dag = c_up.getH()

# Single-site operators for spin-down
# Annihilation operator c_↓
c_down = csr_matrix([
    [0, 0, 1, 0],   # |0> -> 0
    [0, 0, 0, -1],   # |↑> -> 0
    [0, 0, 0, 0],   # |↓> -> |0>
    [0, 0, 0, 0]   # |↑↓> -> -|↑> (anticommutation with c†_↑)
], dtype=np.complex128)

# Creation operator c†_↓
c_down_dag = c_down.getH()

# Number operators
n_up = c_up_dag @ c_up
n_down = c_down_dag @ c_down
n_total_site = n_up + n_down

# Identity operator (4x4 for spinful sites)
I_4 = identity(4, format='csr', dtype=np.complex128)

# Parity operator P = (-1)^n for Jordan-Wigner string
P = csr_matrix([
    [1, 0, 0, 0],   # |0> -> +|0>
    [0, -1, 0, 0],  # |↑> -> -|↑>
    [0, 0, -1, 0],  # |↓> -> -|↓>
    [0, 0, 0, 1]    # |↑↓> -> +|↑↓>
], dtype=np.complex128)

# --- Verification function ---
def verify_spinful_operators():
    """Verify the action of operators on the 4 basis states"""
    print("=== Verification of Spinful Operators ===")
    
    states = [ket_empty, ket_up, ket_down, ket_both]
    state_names = ["|0>", "|↑>", "|↓>", "|↑↓>"]
    
    print("\n1. Spin-up creation operator c†_↑:")
    for i, (state, name) in enumerate(zip(states, state_names)):
        result = c_up_dag @ state
        print(f"   c†_↑ {name} = {result.toarray().flatten()}")
    
    print("\n2. Spin-up annihilation operator c_↑:")
    for i, (state, name) in enumerate(zip(states, state_names)):
        result = c_up @ state
        print(f"   c_↑ {name} = {result.toarray().flatten()}")
    
    print("\n3. Spin-down creation operator c†_↓:")
    for i, (state, name) in enumerate(zip(states, state_names)):
        result = c_down_dag @ state
        print(f"   c†_↓ {name} = {result.toarray().flatten()}")
    
    print("\n4. Spin-down annihilation operator c_↓:")
    for i, (state, name) in enumerate(zip(states, state_names)):
        result = c_down @ state
        print(f"   c_↓ {name} = {result.toarray().flatten()}")
    
    print("\n5. Number operators:")
    for i, (state, name) in enumerate(zip(states, state_names)):
        n_up_result = (n_up @ state).toarray().flatten()
        n_down_result = (n_down @ state).toarray().flatten()
        n_total_result = (n_total_site @ state).toarray().flatten()
        print(f"   n_↑ {name} = {n_up_result}")
        print(f"   n_↓ {name} = {n_down_result}")
        print(f"   n_total {name} = {n_total_result}")
    
    print("\n6. Parity operator P:")
    for i, (state, name) in enumerate(zip(states, state_names)):
        result = (P @ state).toarray().flatten()
        print(f"   P {name} = {result}")
    
    print("\n7. Verify anticommutation on same site:")
    # {c_↑, c†_↓} = 0 and {c_↓, c†_↑} = 0
    anticomm1 = c_up @ c_down_dag + c_down_dag @ c_up
    anticomm2 = c_down @ c_up_dag + c_up_dag @ c_down
    print(f"   {{c_↑, c†_↓}} = \n{anticomm1.toarray()}")
    print(f"   {{c_↓, c†_↑}} = \n{anticomm2.toarray()}")
    
    # {c_↑, c†_↑} = 1 and {c_↓, c†_↓} = 1
    anticomm3 = c_up @ c_up_dag + c_up_dag @ c_up
    anticomm4 = c_down @ c_down_dag + c_down_dag @ c_down
    print(f"   {{c_↑, c†_↑}} = \n{anticomm3.toarray()}")
    print(f"   {{c_↓, c†_↓}} = \n{anticomm4.toarray()}")


# Run verification
verify_spinful_operators()

=== Verification of Spinful Operators ===

1. Spin-up creation operator c†_↑:
   c†_↑ |0> = [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
   c†_↑ |↑> = [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   c†_↑ |↓> = [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
   c†_↑ |↑↓> = [0.+0.j 0.+0.j 0.+0.j 0.+0.j]

2. Spin-up annihilation operator c_↑:
   c_↑ |0> = [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   c_↑ |↑> = [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
   c_↑ |↓> = [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   c_↑ |↑↓> = [0.+0.j 0.+0.j 1.+0.j 0.+0.j]

3. Spin-down creation operator c†_↓:
   c†_↓ |0> = [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
   c†_↓ |↑> = [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]
   c†_↓ |↓> = [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   c†_↓ |↑↓> = [0.+0.j 0.+0.j 0.+0.j 0.+0.j]

4. Spin-down annihilation operator c_↓:
   c_↓ |0> = [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   c_↓ |↑> = [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   c_↓ |↓> = [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
   c_↓ |↑↓> = [ 0.+0.j -1.+0.j  0.+0.j  0.+0.j]

5. Number operators:
   n_↑ |0> = [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   n_↓ |0> = [0.+0.j 0.+0.j 0.+

In [3]:
def full_operator_spinful(L, op, site):
    """
    Create full fermionic operator with Jordan-Wigner string
    Args:
        L: number of sites
        op: single-site operator (c_up, c_up_dag, c_down, c_down_dag, etc.)
        site: site index (0 to L-1)
    """
    if site >= L or site < 0:
        raise ValueError(f"Site index {site} out of range for {L} sites")
    
    result = 1
    for i in range(L):
        if i < site:
            # Jordan-Wigner string: parity operator for all sites to the left
            result = kron(result, P, format='csr')
        elif i == site:
            # Apply the operator at the target site
            result = kron(result, op, format='csr')
        else:
            # Identity for sites to the right
            result = kron(result, I_4, format='csr')
    return result

def creation_operator_up(L, site):
    """Creation operator c†_↑,i with Jordan-Wigner string"""
    return full_operator_spinful(L, c_up_dag, site)

def annihilation_operator_up(L, site):
    """Annihilation operator c_↑,i with Jordan-Wigner string"""
    return full_operator_spinful(L, c_up, site)

def creation_operator_down(L, site):
    """Creation operator c†_↓,i with Jordan-Wigner string"""
    return full_operator_spinful(L, c_down_dag, site)

def annihilation_operator_down(L, site):
    """Annihilation operator c_↓,i with Jordan-Wigner string"""
    return full_operator_spinful(L, c_down, site)

def number_operator_up(L, site):
    """Number operator n_↑,i = c†_↑,i c_↑,i"""
    return full_operator_spinful(L, n_up, site)

def number_operator_down(L, site):
    """Number operator n_↓,i = c†_↓,i c_↓,i"""
    return full_operator_spinful(L, n_down, site)

def number_operator_total_site(L, site):
    """Total number operator n_i = n_↑,i + n_↓,i"""
    return full_operator_spinful(L, n_total_site, site)

def total_number_operator_spinful(L):
    """Total number operator N = Σ_i (n_↑,i + n_↓,i)"""
    N_total = csr_matrix((4**L, 4**L), dtype=np.complex128)
    for i in range(L):
        N_total += number_operator_up(L, i)
        N_total += number_operator_down(L, i)
    return N_total

# --- Check anticommutation relations ---
def check_anticommutation_spinful(L, site_i, site_j):
    """Check anticommutators for spinful fermion operators"""
    # Get all operators
    ci_up = annihilation_operator_up(L, site_i)
    ci_up_dag = creation_operator_up(L, site_i)
    ci_down = annihilation_operator_down(L, site_i)
    ci_down_dag = creation_operator_down(L, site_i)
    
    cj_up = annihilation_operator_up(L, site_j)
    cj_up_dag = creation_operator_up(L, site_j)
    cj_down = annihilation_operator_down(L, site_j)
    cj_down_dag = creation_operator_down(L, site_j)
    
    results = {}
    
    # Same site, same spin: {c_σ,i, c†_σ,i} = 1
    anticomm_same_up = ci_up @ ci_up_dag + ci_up_dag @ ci_up
    anticomm_same_down = ci_down @ ci_down_dag + ci_down_dag @ ci_down
    results[f"{{c_↑,{site_i}, c†_↑,{site_i}}}"] = anticomm_same_up.todense()
    results[f"{{c_↓,{site_i}, c†_↓,{site_i}}}"] = anticomm_same_down.todense()
    
    if site_i != site_j:
        # Different sites, same spin: {c_σ,i, c†_σ,j} = 0
        anticomm_diff_up = ci_up @ cj_up_dag + cj_up_dag @ ci_up
        anticomm_diff_down = ci_down @ cj_down_dag + cj_down_dag @ ci_down
        results[f"{{c_↑,{site_i}, c†_↑,{site_j}}}"] = anticomm_diff_up.todense()
        results[f"{{c_↓,{site_i}, c†_↓,{site_j}}}"] = anticomm_diff_down.todense()
        
        # Different sites, different spins: {c_σ,i, c†_σ',j} = 0
        anticomm_diff_mixed1 = ci_up @ cj_down_dag + cj_down_dag @ ci_up
        anticomm_diff_mixed2 = ci_down @ cj_up_dag + cj_up_dag @ ci_down
        results[f"{{c_↑,{site_i}, c†_↓,{site_j}}}"] = anticomm_diff_mixed1.todense()
        results[f"{{c_↓,{site_i}, c†_↑,{site_j}}}"] = anticomm_diff_mixed2.todense()
    
    # Same site, different spins: {c_σ,i, c†_σ',i} = 0
    anticomm_same_mixed1 = ci_up @ ci_down_dag + ci_down_dag @ ci_up
    anticomm_same_mixed2 = ci_down @ ci_up_dag + ci_up_dag @ ci_down
    results[f"{{c_↑,{site_i}, c†_↓,{site_i}}}"] = anticomm_same_mixed1.todense()
    results[f"{{c_↓,{site_i}, c†_↑,{site_i}}}"] = anticomm_same_mixed2.todense()
    
    return results

# check anticommutation relations for L sites
L = 4
anticom_results = check_anticommutation_spinful(L, 0, 1)
for key, value in anticom_results.items():
    print(f"{key}: {np.round(value, 5)}")

{c_↑,0, c†_↑,0}: [[1.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 ...
 [0.+0.j 0.+0.j 0.+0.j ... 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 1.+0.j]]
{c_↓,0, c†_↓,0}: [[1.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 ...
 [0.+0.j 0.+0.j 0.+0.j ... 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 1.+0.j]]
{c_↑,0, c†_↑,1}: [[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 ...
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]
{c_↓,0, c†_↓,1}: [[0.+0.j 0.+0.j 0.+0.j ... 0.+0.

In [4]:
# Fixed Normal ordering for spinful fermion operators with proper swap counting

def get_operator_priority(op_type, site, spin):
    """
    Get priority for operator ordering:
    1. Creation operators first (lower priority number)
    2. Within creation/annihilation: increasing site number
    3. Within same site: up spin before down spin
    """
    if op_type == 'c_dag':
        base_priority = 0  # Creation operators come first
    else:  # op_type == 'c'
        base_priority = 1000  # Annihilation operators come after
    
    site_priority = site * 10  # Site ordering
    
    if spin == 'up':
        spin_priority = 0
    else:  # spin == 'down'
        spin_priority = 1
    
    return base_priority + site_priority + spin_priority

def count_swaps_to_sort(operators_list):
    """
    Count the number of adjacent swaps needed to sort operators by priority.
    This is equivalent to counting inversions, which gives the correct sign.
    """
    if len(operators_list) <= 1:
        return 0, operators_list
    
    # Create list with priorities and original positions
    indexed_ops = [(get_operator_priority(op[0], op[1], op[2]), i, op) 
                   for i, op in enumerate(operators_list)]
    
    # Use bubble sort to count swaps (this counts exactly the inversions)
    ops_copy = indexed_ops.copy()
    swap_count = 0
    n = len(ops_copy)
    
    for i in range(n):
        for j in range(0, n - i - 1):
            if ops_copy[j][0] > ops_copy[j + 1][0]:
                # Swap needed - this represents an anticommutation
                ops_copy[j], ops_copy[j + 1] = ops_copy[j + 1], ops_copy[j]
                swap_count += 1
    
    # Extract the sorted operator list
    sorted_ops = [op[2] for op in ops_copy]
    
    return swap_count, sorted_ops

def normal_order_operator_list(operators_list):
    """
    Normal order a list of (op_type, site, spin) tuples.
    Returns: (sign, ordered_list) where sign accounts for fermion anticommutations
    """
    if len(operators_list) <= 1:
        return 1.0, operators_list
    
    # Count swaps needed and get sorted list
    swap_count, sorted_ops = count_swaps_to_sort(operators_list)
    
    # Each swap introduces a minus sign due to anticommutation
    sign = (-1) ** swap_count
    
    return sign, sorted_ops

def normal_order_product_spinful(operators_list, L):
    """
    Normal order a product of spinful fermion operators and return the resulting operator.
    
    Args:
        operators_list: List of (op_type, site, spin) tuples
                       op_type: 'c_dag' or 'c'
                       site: integer site index
                       spin: 'up' or 'down'
        L: system size
        
    Returns:
        List of (coefficient, sparse_matrix) tuples representing normal ordered terms
    """
    if len(operators_list) == 0:
        identity_op = identity(4**L, format='csr', dtype=np.complex128)
        return [(1.0, identity_op)]
    
    # Check for repeated operators that give zero (Pauli exclusion)
    op_counts = {}
    for op in operators_list:
        key = (op[0], op[1], op[2])  # (type, site, spin)
        op_counts[key] = op_counts.get(key, 0) + 1 # for a given key, the corresponding value is the count
    
    # If any operator appears more than once, result is zero
    for count in op_counts.values():
        if count > 1:
            zero_op = csr_matrix((4**L, 4**L), dtype=np.complex128)
            return [(0.0, zero_op)]
    
    # Get the normal ordered form with correct sign
    sign, ordered_ops = normal_order_operator_list(operators_list)
    
    # Construct the full operator by multiplying individual operators
    result_op = identity(4**L, format='csr', dtype=np.complex128)
    
    for op_type, site, spin in ordered_ops:
        if op_type == 'c_dag' and spin == 'up':
            op = creation_operator_up(L, site)
        elif op_type == 'c_dag' and spin == 'down':
            op = creation_operator_down(L, site)
        elif op_type == 'c' and spin == 'up':
            op = annihilation_operator_up(L, site)
        elif op_type == 'c' and spin == 'down':
            op = annihilation_operator_down(L, site)
        else:
            raise ValueError(f"Invalid operator: {op_type}, {spin}")
        
        result_op = result_op @ op
    
    return [(complex(sign), result_op)]

def print_operator_sequence(operators_list):
    """Print the operator sequence in readable form"""
    op_strings = []
    for op_type, site, spin in operators_list:
        if op_type == 'c_dag':
            symbol = f"c†_{spin[0]},{site}"
        else:
            symbol = f"c_{spin[0]},{site}"
        op_strings.append(symbol)
    return " ".join(op_strings)

def test_normal_ordering_comprehensive():
    """Comprehensive test of the normal ordering function"""
    L = 4
    print("=== Comprehensive Normal Ordering Tests ===\n")
    
    # Test 1: Already in normal order
    print("Test 1: c†_u,0 c†_d,0 (already normal ordered)")
    ops = [('c_dag', 0, 'up'), ('c_dag', 0, 'down')]
    print(f"Original: {print_operator_sequence(ops)}")
    sign, ordered = normal_order_operator_list(ops)
    print(f"Normal ordered: {sign:+.0f} × {print_operator_sequence(ordered)}")
    print(f"Swaps needed: 0 (already ordered)\n")
    
    # Test 2: Simple swap - spin ordering
    print("Test 2: c†_d,0 c†_u,0 -> need to swap spins")
    ops = [('c_dag', 0, 'down'), ('c_dag', 0, 'up')]
    print(f"Original: {print_operator_sequence(ops)}")
    sign, ordered = normal_order_operator_list(ops)
    print(f"Normal ordered: {sign:+.0f} × {print_operator_sequence(ordered)}")
    print(f"Swaps needed: 1 -> sign = (-1)^1 = -1\n")
    
    # Test 3: Site ordering
    print("Test 3: c†_u,1 c†_u,0 -> need to swap sites")
    ops = [('c_dag', 1, 'up'), ('c_dag', 0, 'up')]
    print(f"Original: {print_operator_sequence(ops)}")
    sign, ordered = normal_order_operator_list(ops)
    print(f"Normal ordered: {sign:+.0f} × {print_operator_sequence(ordered)}")
    print(f"Swaps needed: 1 -> sign = (-1)^1 = -1\n")
    
    # Test 4: Creation vs annihilation
    print("Test 4: c_u,0 c†_u,1 -> creation should come first")
    ops = [('c', 0, 'up'), ('c_dag', 1, 'up')]
    print(f"Original: {print_operator_sequence(ops)}")
    sign, ordered = normal_order_operator_list(ops)
    print(f"Normal ordered: {sign:+.0f} × {print_operator_sequence(ordered)}")
    print(f"Swaps needed: 1 -> sign = (-1)^1 = -1\n")
    
    # Test 5: Multiple swaps
    print("Test 5: c_d,1 c†_d,0 c_u,0 c†_u,1 -> multiple reorderings")
    ops = [('c', 1, 'down'), ('c_dag', 0, 'down'), ('c', 0, 'up'), ('c_dag', 1, 'up')]
    print(f"Original: {print_operator_sequence(ops)}")
    
    # Let's manually trace the swaps:
    # Start: c_d,1 c†_d,0 c_u,0 c†_u,1
    # Target order: c†_u,0 c†_d,0 c†_u,1 c_u,0 c_d,1
    # But we need c†_u,1 not c†_u,0 twice, so:
    # Target: c†_u,1 c†_d,0 c_u,0 c_d,1
    
    sign, ordered = normal_order_operator_list(ops)
    print(f"Normal ordered: {sign:+.0f} × {print_operator_sequence(ordered)}")
    
    # Count swaps manually to verify
    swap_count, _ = count_swaps_to_sort(ops)
    print(f"Swaps needed: {swap_count} -> sign = (-1)^{swap_count} = {(-1)**swap_count}\n")
    
    # Test 6: Complex example with many operators
    print("Test 6: Complex example - c_d,2 c†_u,0 c_u,1 c†_d,1 c†_u,2")
    ops = [('c', 2, 'down'), ('c_dag', 0, 'up'), ('c', 1, 'up'), ('c_dag', 1, 'down'), ('c_dag', 2, 'up')]
    print(f"Original: {print_operator_sequence(ops)}")
    sign, ordered = normal_order_operator_list(ops)
    print(f"Normal ordered: {sign:+.0f} × {print_operator_sequence(ordered)}")
    swap_count, _ = count_swaps_to_sort(ops)
    print(f"Swaps needed: {swap_count} -> sign = (-1)^{swap_count} = {(-1)**swap_count}\n")
    
    # Test 7: Repeated operator (should give zero)
    print("Test 7: c†_u,0 c†_u,0 -> should be zero (Pauli exclusion)")
    ops = [('c_dag', 0, 'up'), ('c_dag', 0, 'up')]
    print(f"Original: {print_operator_sequence(ops)}")
    result = normal_order_product_spinful(ops, L)
    print(f"Result coefficient: {result[0][0]} (should be 0)\n")

def verify_anticommutation_with_normal_ordering():
    """Verify that our normal ordering respects anticommutation relations"""
    L = 3
    print("=== Verification: Normal Ordering vs Direct Anticommutation ===\n")
    
    # Test: c†_u,0 c†_d,1 vs c†_d,1 c†_u,0
    print("Comparing c†_u,0 c†_d,1 vs c†_d,1 c†_u,0:")
    
    # First order
    ops1 = [('c_dag', 0, 'up'), ('c_dag', 1, 'down')]
    result1 = normal_order_product_spinful(ops1, L)
    op1 = result1[0][1]
    coeff1 = result1[0][0]
    
    # Second order (swapped)
    ops2 = [('c_dag', 1, 'down'), ('c_dag', 0, 'up')]
    result2 = normal_order_product_spinful(ops2, L)
    op2 = result2[0][1]
    coeff2 = result2[0][0]

    print(op1.toarray())
    print(op2.toarray())
    
    print(f"First:  {print_operator_sequence(ops1)} -> coeff = {np.real(coeff1)}")
    print(f"Second: {print_operator_sequence(ops2)} -> coeff = {np.real(coeff2)}")
    
    # Check if op1 = -op2 (they should be negatives of each other)
    diff = op1 + coeff2 * op2
    is_anticommuting = np.allclose(diff.toarray(), 0)
    print(f"Are they anticommuting? {is_anticommuting}")
    print(f"Coefficient ratio: {np.real(coeff2/coeff1)} (should be -1)\n")

# Run comprehensive tests
test_normal_ordering_comprehensive()
verify_anticommutation_with_normal_ordering()

=== Comprehensive Normal Ordering Tests ===

Test 1: c†_u,0 c†_d,0 (already normal ordered)
Original: c†_u,0 c†_d,0
Normal ordered: +1 × c†_u,0 c†_d,0
Swaps needed: 0 (already ordered)

Test 2: c†_d,0 c†_u,0 -> need to swap spins
Original: c†_d,0 c†_u,0
Normal ordered: -1 × c†_u,0 c†_d,0
Swaps needed: 1 -> sign = (-1)^1 = -1

Test 3: c†_u,1 c†_u,0 -> need to swap sites
Original: c†_u,1 c†_u,0
Normal ordered: -1 × c†_u,0 c†_u,1
Swaps needed: 1 -> sign = (-1)^1 = -1

Test 4: c_u,0 c†_u,1 -> creation should come first
Original: c_u,0 c†_u,1
Normal ordered: -1 × c†_u,1 c_u,0
Swaps needed: 1 -> sign = (-1)^1 = -1

Test 5: c_d,1 c†_d,0 c_u,0 c†_u,1 -> multiple reorderings
Original: c_d,1 c†_d,0 c_u,0 c†_u,1
Normal ordered: +1 × c†_d,0 c†_u,1 c_u,0 c_d,1
Swaps needed: 4 -> sign = (-1)^4 = 1

Test 6: Complex example - c_d,2 c†_u,0 c_u,1 c†_d,1 c†_u,2
Original: c_d,2 c†_u,0 c_u,1 c†_d,1 c†_u,2
Normal ordered: +1 × c†_u,0 c†_d,1 c†_u,2 c_u,1 c_d,2
Swaps needed: 6 -> sign = (-1)^6 = 1

Test 7: c†

In [5]:
# OSS.: FERMIONS COME IN PAIRS IN THE ETA SCARS SO THAT THEY CAN BE EFFECTIVELY TREATED LIKE BOSONS WHEN COMPUTING THE PARTIAL TRACE

'''
def ptrace_sparse_normal_ordered(dm_sparse, keep, trace, L):
    """
    Compute partial trace using normal-ordered fermion operators.
    
    Args:
        dm_sparse: Full density matrix in sparse format
        keep: List of site indices to keep  
        trace: List of site indices to trace out
        L: Total number of sites
        
    Returns:
        Reduced density matrix over kept sites
    """
    if not issparse(dm_sparse):
        raise ValueError("dm_sparse must be a scipy.sparse matrix")
    
    dims = [4] * L  # 4 states per site for spinful fermions
    d_keep = 4 ** len(keep)
    
    # Convert to COO format for element-wise processing
    dm_sparse = dm_sparse.tocoo()
    
    # Storage for reduced matrix
    data = []
    row_idx = []
    col_idx = []
    
    def basis_state_to_operators(state_idx, sites):
        """Convert a basis state index to creation operators from vacuum"""
        state_config = idx_to_subsys(state_idx, [4] * len(sites))
        operators = []
        
        for i, site in enumerate(sites):
            local_state = state_config[i]
            # Convert local state to operators: |0⟩, |↑⟩, |↓⟩, |↑↓⟩
            if local_state == 1:  # |↑⟩
                operators.append(('c_dag', site, 'up'))
            elif local_state == 2:  # |↓⟩  
                operators.append(('c_dag', site, 'down'))
            elif local_state == 3:  # |↑↓⟩
                operators.append(('c_dag', site, 'up'))
                operators.append(('c_dag', site, 'down'))
            # local_state == 0 corresponds to vacuum, no operators needed
                
        return operators
    
    def compute_traced_expectation(bra_ops, ket_ops, trace_sites):
        """
        Compute ⟨trace_bra|operator_sequence|trace_ket⟩ using normal ordering.
        This gives the coefficient when we trace out the specified sites.
        """
        # For partial trace: ⟨trace_state|ket_ops† bra_ops|trace_state⟩  
        # We need to compute: ⟨0|bra_ops† ket_ops|0⟩ for each trace state
        
        total_expectation = 0.0
        
        # Sum over all possible states of traced subsystem
        for trace_state_idx in range(4 ** len(trace_sites)):
            trace_config = idx_to_subsys(trace_state_idx, [4] * len(trace_sites))
            
            # Create the trace state operators
            trace_bra_ops = basis_state_to_operators(trace_state_idx, trace_sites)
            trace_ket_ops = trace_bra_ops.copy()  # Same state for bra and ket
            
            # Build full operator sequence: 
            # ⟨trace_bra| ket_ops† bra_ops |trace_ket⟩
            # = ⟨0| trace_bra_ops† ket_ops† bra_ops trace_ket_ops |0⟩
            
            full_operator_sequence = []
            
            # Add annihilation operators for trace bra (reversed order for hermitian conjugate)
            for op_type, site, spin in reversed(trace_bra_ops):
                if op_type == 'c_dag':
                    full_operator_sequence.append(('c', site, spin))
            
            # Add hermitian conjugate of ket operators (creation → annihilation, reverse order)
            for op_type, site, spin in reversed(ket_ops):
                if op_type == 'c_dag':
                    full_operator_sequence.append(('c', site, spin))
                else:  # op_type == 'c'
                    full_operator_sequence.append(('c_dag', site, spin))
            
            # Add bra operators
            full_operator_sequence.extend(bra_ops)
            
            # Add creation operators for trace ket
            full_operator_sequence.extend(trace_ket_ops)
            
            # Normal order and compute vacuum expectation value
            try:
                result = normal_order_product_spinful(full_operator_sequence, L)
                if len(result) > 0:
                    coeff, matrix = result[0]
                    
                    # For vacuum expectation value, we need ⟨0|O|0⟩
                    # This is non-zero only if the normal-ordered operator is identity
                    vacuum_state = vacuum_state_spinful(L)
                    expectation_val = (vacuum_state.getH() @ matrix @ vacuum_state)[0,0]
                    
                    total_expectation += expectation_val
                    
            except Exception as e:
                # Handle cases where normal ordering fails (e.g., invalid operators)
                continue
                
        return total_expectation
    
    # Process each non-zero element of the density matrix
    for i, j, val in tqdm(zip(dm_sparse.row, dm_sparse.col, dm_sparse.data)):
        # Convert full space indices to site configurations
        full_bra_config = idx_to_subsys(i, dims)  # ⟨i|
        full_ket_config = idx_to_subsys(j, dims)  # |j⟩
        
        # Extract configurations for kept and traced sites
        keep_bra_config = [full_bra_config[site] for site in keep]
        keep_ket_config = [full_ket_config[site] for site in keep]
        trace_bra_config = [full_bra_config[site] for site in trace]
        trace_ket_config = [full_ket_config[site] for site in trace]
        
        # Convert to operator sequences
        keep_bra_ops = basis_state_to_operators(
            sum(keep_bra_config[k] * (4**(len(keep)-1-k)) for k in range(len(keep))), 
            keep
        )
        keep_ket_ops = basis_state_to_operators(
            sum(keep_ket_config[k] * (4**(len(keep)-1-k)) for k in range(len(keep))), 
            keep
        )
        trace_bra_ops = basis_state_to_operators(
            sum(trace_bra_config[k] * (4**(len(trace)-1-k)) for k in range(len(trace))), 
            trace
        )
        trace_ket_ops = basis_state_to_operators(
            sum(trace_ket_config[k] * (4**(len(trace)-1-k)) for k in range(len(trace))), 
            trace
        )
        
        # Compute traced expectation value using normal ordering
        if trace_bra_config == trace_ket_config:  # Only diagonal elements in trace space contribute
            traced_coeff = compute_traced_expectation(trace_bra_ops, trace_ket_ops, trace)
            
            if abs(traced_coeff) > 1e-14:  # Only include non-zero contributions
                # Compute reduced space indices
                i_red = sum(keep_bra_config[k] * (4**(len(keep)-1-k)) for k in range(len(keep)))
                j_red = sum(keep_ket_config[k] * (4**(len(keep)-1-k)) for k in range(len(keep)))
                
                # Add to reduced matrix
                data.append(val * traced_coeff)
                row_idx.append(i_red)
                col_idx.append(j_red)
    
    return coo_matrix((data, (row_idx, col_idx)), shape=(d_keep, d_keep)).tocsr()

def idx_to_subsys(idx, dims):
    """Convert flat index to subsystem indices (reuse from your existing code)"""
    subsys = []
    for d in reversed(dims):
        subsys.append(idx % d)
        idx //= d
    return np.array(subsys[::-1])

    '''

'\ndef ptrace_sparse_normal_ordered(dm_sparse, keep, trace, L):\n    """\n    Compute partial trace using normal-ordered fermion operators.\n    \n    Args:\n        dm_sparse: Full density matrix in sparse format\n        keep: List of site indices to keep  \n        trace: List of site indices to trace out\n        L: Total number of sites\n        \n    Returns:\n        Reduced density matrix over kept sites\n    """\n    if not issparse(dm_sparse):\n        raise ValueError("dm_sparse must be a scipy.sparse matrix")\n    \n    dims = [4] * L  # 4 states per site for spinful fermions\n    d_keep = 4 ** len(keep)\n    \n    # Convert to COO format for element-wise processing\n    dm_sparse = dm_sparse.tocoo()\n    \n    # Storage for reduced matrix\n    data = []\n    row_idx = []\n    col_idx = []\n    \n    def basis_state_to_operators(state_idx, sites):\n        """Convert a basis state index to creation operators from vacuum"""\n        state_config = idx_to_subsys(state_idx, [4

In [6]:
def innermost_adjacent_indices(L, block_size):
    """
    Returns the indices of the innermost adjacent block of given size.
    For even L, the block is centered in the middle.
    """
    start = (L - block_size) // 2
    return list(range(start, start + block_size))

def all_adjacent_indices(L, block_size, bc):
    """
    Returns a list of all possible adjacent blocks of given size.
    Each block is represented as a list of indices.
    
    Args:
        L: system size  
        block_size: size of the block
        bc: boundary conditions ('obc' or 'pbc')
    """
    blocks = []
    
    if bc == 'obc':
        # Original OBC implementation
        for start in range(L - block_size + 1):
            blocks.append(list(range(start, start + block_size)))
    
    elif bc == 'pbc':
        # PBC implementation - sites wrap around
        for start in range(L):
            block = []
            for i in range(block_size):
                block.append((start + i) % L)  # Wrap around using modulo
            blocks.append(block)
    
    else:
        raise ValueError("Boundary condition must be 'obc' or 'pbc'")
    
    return blocks

def ptrace_sparse(dm_sparse, keep, dims):
    """
    Compute the partial trace over arbitrary subsystems using sparse matrix operations.

    Args:
        dm_sparse (scipy.sparse matrix): Full density matrix of shape (D, D), where D = product(dims)
        keep (list of int): Subsystems to keep (indices, 0-indexed)
        dims (list of int): List of subsystem dimensions, e.g., [2]*n for n qubits

    Returns:
        scipy.sparse.csr_matrix: Reduced density matrix over kept subsystems
    """
    if not issparse(dm_sparse):
        raise ValueError("dm_sparse must be a scipy.sparse matrix")
    n = len(dims)
    D = np.prod(dims)
    if dm_sparse.shape != (D, D):
        raise ValueError("Density matrix shape does not match dims")
    trace = [i for i in range(n) if i not in keep]
    d_keep = np.prod([dims[i] for i in keep])
    # Prepare output
    data = []
    row_idx = []
    col_idx = []

    # Precompute bit masks
    #def idx_to_bits(idx):
    #    return np.array(list(np.binary_repr(idx, width=n))).astype(int)

    def idx_to_subsys(idx, dims):
    #Convert flat index to tuple of subsystem indices for arbitrary dims.
        subsys = []
        for d in reversed(dims):
            subsys.append(idx % d)
            idx //= d
        return np.array(subsys[::-1])

    

    dm_sparse = dm_sparse.tocoo()

    for i, j, val in tqdm(zip(dm_sparse.row, dm_sparse.col, dm_sparse.data)):
        bi = idx_to_subsys(i, dims)
        bj = idx_to_subsys(j, dims)

        if np.all(bi[trace] == bj[trace]):
            i_red = 0
            j_red = 0
            for k, pos in enumerate(keep):
                i_red = i_red * dims[pos] + bi[pos]
                j_red = j_red * dims[pos] + bj[pos]

            data.append(val)
            row_idx.append(i_red)
            col_idx.append(j_red)

    
    return coo_matrix((data, (row_idx, col_idx)), shape=(d_keep, d_keep)).tocsr()

def ptrace_sparse_parallel(dm_sparse, keep, dims, n_jobs=-1): # njobs to be removed if not using joblib
    """
    Compute the partial trace over arbitrary subsystems using sparse matrix operations.
    Parallelized over nonzero elements.
    """
    if not issparse(dm_sparse):
        raise ValueError("dm_sparse must be a scipy.sparse matrix")
    n = len(dims)
    D = np.prod(dims)
    if dm_sparse.shape != (D, D):
        raise ValueError("Density matrix shape does not match dims")
    trace = [i for i in range(n) if i not in keep]
    d_keep = np.prod([dims[i] for i in keep])


    def idx_to_subsys(idx, dims):
    #Convert flat index to tuple of subsystem indices for arbitrary dims.
        subsys = []
        for d in reversed(dims):
            subsys.append(idx % d)
            idx //= d
        return np.array(subsys[::-1])

    
    dm_sparse = dm_sparse.tocoo()

    def process_entry(i,j,val):
        bi = idx_to_subsys(i, dims)
        bj = idx_to_subsys(j, dims)

        if np.all(bi[trace] == bj[trace]):
            i_red = 0
            j_red = 0
            for k, pos in enumerate(keep):
                i_red = i_red * dims[pos] + bi[pos]
                j_red = j_red * dims[pos] + bj[pos]
            return (val, i_red, j_red)
        else:
            return None
        
    results = Parallel(n_jobs=n_jobs, prefer="processes")(
        delayed(process_entry)(i, j, val)
        for i, j, val in tqdm(zip(dm_sparse.row, dm_sparse.col, dm_sparse.data))
    )
    results = [r for r in results if r is not None]

    '''entries = zip(psi_sparse.row, psi_sparse.col, psi_sparse.data)
    results = []
    with ThreadPoolExecutor() as executor:
        for res in executor.map(process_entry, entries):
            if res is not None:
                results.append(res)'''
    
    if results:
        data, row_idx, col_idx = zip(*results)
    else:
        data, row_idx, col_idx = [], [], []

    return coo_matrix((data, (row_idx, col_idx)), shape=(d_keep, d_keep)).tocsr()

# ...existing code...

def fermionic_sign(full_config, keep, trace):
    """
    Compute the fermionic sign for a given occupation configuration
    when reordering sites to bring 'keep' sites to the front.
    full_config: array of site occupations (length L)
    keep: list of kept site indices
    trace: list of traced site indices
    """
    # Build the permutation that brings keep+trace to original order
    perm = keep + trace
    # For each pair (i<j), if perm[i] > perm[j] and both are occupied, count a swap
    sign = 1
    occ = np.array(full_config)
    # Only need to count swaps between occupied sites
    for i in range(len(perm)):
        for j in range(i+1, len(perm)):
            if perm[i] > perm[j] and occ[perm[i]] and occ[perm[j]]:
                sign *= -1
    return sign

def ptrace_sparse_fermionic(dm_sparse, keep, dims):
    """
    Partial trace for fermions, including the correct sign.
    """
    if not issparse(dm_sparse):
        raise ValueError("dm_sparse must be a scipy.sparse matrix")
    n = len(dims)
    D = np.prod(dims)
    if dm_sparse.shape != (D, D):
        raise ValueError("Density matrix shape does not match dims")
    trace = [i for i in range(n) if i not in keep]
    d_keep = np.prod([dims[i] for i in keep])
    data = []
    row_idx = []
    col_idx = []

    dm_sparse = dm_sparse.tocoo()

    def idx_to_subsys(idx, dims):
        subsys = []
        for d in reversed(dims):
            subsys.append(idx % d)
            idx //= d
        return np.array(subsys[::-1])

    for i, j, val in tqdm(zip(dm_sparse.row, dm_sparse.col, dm_sparse.data)):
        bi = idx_to_subsys(i, dims)
        bj = idx_to_subsys(j, dims)
        # Only keep terms where traced sites match
        if np.all(bi[trace] == bj[trace]):
            # Compute fermionic sign for this configuration
            sign_i = fermionic_sign(bi, keep, trace)
            sign_j = fermionic_sign(bj, keep, trace)
            sign = sign_i * sign_j
            # Map to reduced indices
            i_red = 0
            j_red = 0
            for k, pos in enumerate(keep):
                i_red = i_red * dims[pos] + bi[pos]
                j_red = j_red * dims[pos] + bj[pos]
            data.append(val * sign)
            row_idx.append(i_red)
            col_idx.append(j_red)

    return coo_matrix((data, (row_idx, col_idx)), shape=(d_keep, d_keep)).tocsr()

def ptrace_sparse_fermionic_parallel(dm_sparse, keep, dims, n_jobs=-1):
    """
    Parallel partial trace for fermions, including the correct sign.
    """
    if not issparse(dm_sparse):
        raise ValueError("dm_sparse must be a scipy.sparse matrix")
    n = len(dims)
    D = np.prod(dims)
    if dm_sparse.shape != (D, D):
        raise ValueError("Density matrix shape does not match dims")
    trace = [i for i in range(n) if i not in keep]
    d_keep = np.prod([dims[i] for i in keep])

    def idx_to_subsys(idx, dims):
        subsys = []
        for d in reversed(dims):
            subsys.append(idx % d)
            idx //= d
        return np.array(subsys[::-1])

    dm_sparse = dm_sparse.tocoo()

    def process_entry_fermionic(i, j, val):
        bi = idx_to_subsys(i, dims)
        bj = idx_to_subsys(j, dims)

        if np.all(bi[trace] == bj[trace]):
            # Compute fermionic sign for this configuration
            sign_i = fermionic_sign(bi, keep, trace)
            sign_j = fermionic_sign(bj, keep, trace)
            sign = sign_i * sign_j
            # Map to reduced indices
            i_red = 0
            j_red = 0
            for k, pos in enumerate(keep):
                i_red = i_red * dims[pos] + bi[pos]
                j_red = j_red * dims[pos] + bj[pos]
            return (val * sign, i_red, j_red)
        else:
            return None
        
    results = Parallel(n_jobs=n_jobs, prefer="processes")(
        delayed(process_entry_fermionic)(i, j, val)
        for i, j, val in tqdm(zip(dm_sparse.row, dm_sparse.col, dm_sparse.data))
    )
    results = [r for r in results if r is not None]
    
    if results:
        data, row_idx, col_idx = zip(*results)
    else:
        data, row_idx, col_idx = [], [], []

    return coo_matrix((data, (row_idx, col_idx)), shape=(d_keep, d_keep)).tocsr()

# ...existing code...

def ee_sparse(dm_sparse, L):
    """
    Computes the entanglement entropy of a state using sparse matrices in parallel.
    The state is assumed to be a vector in the Hilbert space of L qubits.
    """
    rhoA = ptrace_sparse(dm_sparse, list(range(L // 2)), [4] * L)
    eigvals = np.linalg.eigvalsh(rhoA.toarray())
    return -np.sum(eigvals * np.log(eigvals + 1e-12)).real  # Add small value to avoid log(0)

def ee_sparse_parallel(dm_sparse, L, n_jobs=-1):
    """
    Computes the entanglement entropy of a state using sparse matrices in parallel.
    The state is assumed to be a vector in the Hilbert space of L qubits.
    """
    rhoA = ptrace_sparse_parallel(dm_sparse, list(range(L // 2)), [4] * L, n_jobs=n_jobs)
    eigvals = np.linalg.eigvalsh(rhoA.toarray())
    return -np.sum(eigvals * np.log(eigvals + 1e-12)).real  # Add small value to avoid log(0)

def rdm_qutip(state, L, keep_qubits):
    rho = np.outer(state, state.conj())
    rho_qobj = Qobj(rho, dims=[[4] * L, [4] * L])
    rdm = ptrace(rho_qobj, keep_qubits)
    rdm_mat = rdm.full()
    eigvals = np.linalg.eigvalsh(rdm_mat)
    min_eigval = np.min(eigvals)
    # Rank: count nonzero eigenvalues (with tolerance)
    rank = np.sum(eigvals > 1e-12)
    return rdm, min_eigval, rank

def ee_qutip(state, L):
    rho = np.outer(state, state.conj())
    rho_qobj = Qobj(rho, dims=[[4] * L, [4] * L])
    rhoA = ptrace(rho_qobj, list(range(L//2)))
    return entropy_vn(rhoA)

In [7]:
# eta pairing states

# --- vacuum/initial state ---
def vacuum_state_spinful(L):
    """Create vacuum state |0>^⊗L for L sites"""
    state = ket_empty
    for _ in range(L-1):
        state = kron(state, ket_empty, format='csr')
    return state

# --- eta^+ operator ---
def eta_plus(L):
    etap = csr_matrix((4**L, 4**L), dtype=complex)
    for k in range(L):
        phase = np.exp(1j * np.pi * k)
        eplus = creation_operator_up(L, k).dot(creation_operator_down(L, k))
        etap += phase * eplus

    return etap

def t_plus(L, bc = 'periodic'):
    tp = csr_matrix((4**L, 4**L), dtype=complex)
    for k in range(L):
        phase = np.exp(1j * np.pi * k)
        tplus = creation_operator_up(L, k).dot(creation_operator_up(L, (k+1) % L))
        tp += phase * tplus

    return tp


def s_plus(L, bc = 'periodic'):
    sp = csr_matrix((4**L, 4**L), dtype=complex)
    for k in range(L):
        phase = np.exp(1j * np.pi * k)
        splus = (creation_operator_up(L, k).dot(creation_operator_down(L, (k+1) % L)) - creation_operator_down(L, k).dot(creation_operator_up(L, (k+1) % L)))
        sp += phase * splus

    return sp


# --- eta_tower states ---
def eta_tower(n, L):
    norm = np.sqrt(factorial(L-n) / (factorial(n) * factorial(L)))
    eta = eta_plus(L)
    state = vacuum_state_spinful(L)
    for _ in range(1,n+1):
        state = eta.dot(state)
        
    return norm * state

# --- t_tower states ---
def t_tower(m, n, L, bc='periodic'):
    norm = np.sqrt(factorial(L-n) / (factorial(n) * factorial(L)))
    eta = eta_plus(L)
    t = t_plus(L, bc)
    state = vacuum_state_spinful(L)
    
    #print(f"Initial vacuum norm: {np.linalg.norm(state.toarray())}")
    
    # Apply η⁺ operators
    for i in range(1, n+1):
        state = eta.dot(state)
        #print(f"After η⁺^{i}, norm: {np.linalg.norm(state.toarray())}")
    
    #print(f"Before t⁺ application, non-zero elements: {np.sum(np.abs(state.toarray()) > 1e-12)}")
    
    # Apply t⁺ operators
    for i in range(1, m+1):
        #old_norm = np.linalg.norm(state.toarray())
        state = t.dot(state)
        #new_norm = np.linalg.norm(state.toarray())
        #print(f"After t⁺^{i}, norm: {old_norm} -> {new_norm}")
        #print(f"Non-zero elements: {np.sum(np.abs(state.toarray()) > 1e-12)}")
        
        #if new_norm < 1e-12:
        #    print(f"State became zero after t⁺^{i}!")
        #    return csr_matrix((len(state), 1))
    
    return norm * state

# --- s_tower states ---
def s_tower(n, L):
    norm = np.sqrt(factorial(L-n) / (factorial(n) * factorial(L)))
    eta = eta_plus(L)
    s = s_plus(L)
    state = vacuum_state_spinful(L)
    for _ in range(1,n+1):
        state = eta.dot(state)
    state = s.dot(state)
        
    return norm * state

# test

L = 12
n = 6
m = 2
state = t_tower(m, n, L).toarray()
# Check if any component is non-zero
has_nonzero = np.any(np.abs(state) > 1e-12)
num_nonzero = np.sum(np.abs(state) > 1e-12)
max_component = np.max(np.abs(state))

print(f"Eta tower state for L={L}, n={n}:")
print(f"Has any non-zero components: {has_nonzero}")
print(f"Number of non-zero components: {num_nonzero}")
print(f"Total components: {len(state)}")
print(f"Maximum component magnitude: {max_component}")
print(f"State norm: {np.linalg.norm(state)}")

# Show actual non-zero components
nonzero_indices = np.where(np.abs(state) > 1e-12)[0]
if len(nonzero_indices) > 0:
    print(f"\nFirst few non-zero components:")
    for i in nonzero_indices[:10]:  # Show first 10
        print(f"  Index {i}: {state[i]}")

Eta tower state for L=12, n=6:
Has any non-zero components: True
Number of non-zero components: 1512
Total components: 16777216
Maximum component magnitude: 0.0657951694959769
State norm: 2.5584085962673235

First few non-zero components:
  Index 352255: [-0.06579517+6.93033708e-16j]
  Index 382975: [0.06579517-6.84976124e-16j]
  Index 390655: [-0.06579517+6.76918539e-16j]
  Index 392575: [0.06579517-6.68860955e-16j]
  Index 393055: [-0.06579517+6.6080337e-16j]
  Index 393175: [0.06579517-6.52745786e-16j]
  Index 393205: [-0.06579517+4.10936803e-16j]
  Index 874495: [-0.06579517+6.76918539e-16j]
  Index 882175: [0.06579517-6.68860955e-16j]
  Index 884095: [-0.06579517+6.6080337e-16j]


In [35]:
'''def number_op(site, spin, L):
    """
    Number operator n_{site,spin} in the full 2**(2L) Hilbert space.
    spin=0 for up, 1 for down.
    """
    dim = 4**L
    # Which bit corresponds to (site,spin)
    mode_index = 2*site + spin
    # Diagonal: 1 if bit is occupied, 0 if not
    diag = np.array([( (b >> mode_index) & 1 ) for b in range(dim)], dtype=float)
    return diags(diag, format='csr')

def parity_operator(L):
    """
    Build the global fermion-number parity operator P = (-1)^N.
    """
    P = identity(4**L, format='csr')
    for site in range(L):
        for spin in (0, 1):
            n = number_op(site, spin, L)
            P = P @ (identity(P.shape[0], format='csr') - 2*n)
    return P'''

def parity_operator(L):
    """
    Build the global fermion-number parity operator P = (-1)^N
    using proper fermionic operators with Jordan-Wigner strings.
    """
    # Start with identity
    P = identity(4**L, format='csr', dtype=np.complex128)
    
    # Add contribution from each fermion mode
    for site in range(L):
        n_up_op = number_operator_up(L, site)
        n_down_op = number_operator_down(L, site)
        
        # P *= (-1)^(n_up + n_down) = (-1)^n_up * (-1)^n_down
        P = P @ (identity(4**L, format='csr') - 2*n_up_op)
        P = P @ (identity(4**L, format='csr') - 2*n_down_op)
    
    return P

def parity_operator_correct(L):
    """P = (-1)^N using tensor product of single-site parity operators"""
    # Single-site parity: diag(+1, -1, -1, +1) for |0⟩, |↑⟩, |↓⟩, |↑↓⟩
    P_site = diags([1, -1, -1, 1], format='csr', dtype=np.complex128)
    
    result = P_site
    for _ in range(L-1):
        result = kron(result, P_site, format='csr')
    
    return result

def parity_of_state(state, P):
    """
    Compute <psi|P|psi> for normalized state vector.
    For eigenstates of P, this will be ±1.
    """
    return np.vdot(state, P @ state)

def parity_of_state_sparse(state, P):
    """
    Compute <psi|P|psi> for normalized state vector using sparse operations.
    Accepts both dense arrays and sparse column vectors.
    """
    # Convert state to sparse column vector if needed
    if isinstance(state, csr_matrix):
        psi = state
    else:
        # Dense array -> sparse column vector
        vec = np.asarray(state).ravel()
        psi = csr_matrix(vec.reshape(-1, 1), dtype=np.complex128)
    
    # Compute <ψ|P|ψ> = ψ^H @ P @ ψ (all sparse operations)
    result = psi.conj().T @ P @ psi
    return result[0, 0].real  # Extract scalar value

# Example: 2-site Hubbard system
L = 2
P_op = parity_operator_correct(L)

# Example basis vector: single up electron at site 0
dim = 4**L
state = np.zeros(dim, dtype=complex)
state[1] = 1.0
print("Parity =", parity_of_state_sparse(state, P_op))

Parity = -1.0


In [9]:
# parity function definition + check for full eta,s,t-states - it should be even

In [36]:
# ---- Parity utilities for 4^L basis (local states: 0, up, down, updown) ----

def _num_particles_local(label):
    # local label in {0,1,2,3} -> particle count
    # 0: |0>, 1: |↑>, 2: |↓>, 3: |↑↓>
    return (label == 1) + (label == 2) + 2 * (label == 3)  # works because booleans -> ints

def _particle_number_from_index(idx, L):
    # decode idx in base-4 (little-endian sites) and count particles
    N = 0
    for _ in range(L):
        N += _num_particles_local(idx % 4)
        idx //= 4
    return N

def parity_expectation(state, L, atol=1e-12):
    """
    Compute <P> for a state in the 4^L occupation basis without constructing P.
    Accepts dense shape (4**L,) or (4**L,1) arrays, or CSR column vector.
    """
    # get dense 1D amplitude array
    if isinstance(state, csr_matrix):
        vec = state.toarray().ravel()
    else:
        vec = np.asarray(state).ravel()

    # fast path: if mostly sparse, iterate only on nonzeros
    nz = np.nonzero(np.abs(vec) > atol)[0]
    if nz.size == 0:
        return 0.0

    expP = 0.0
    for i in nz:
        Ni = _particle_number_from_index(i, L)
        expP += (np.abs(vec[i])**2) * (1.0 if (Ni % 2 == 0) else -1.0)
    return float(np.real_if_close(expP))

# ---------------- Demo on your towers (unchanged tower builders) ----------------
# assuming: vacuum_state_spinful, eta_plus, t_plus, s_plus, eta_tower, t_tower, s_tower exist

# Example parameters
L = 12
n = 6
m = 2

eta_state   = eta_tower(n, L)  # (eta^+)^n |0>
t_state     = t_tower(m, n, L) # (t^+)^m (eta^+)^n |0>
t_state = t_state/sparse_norm(t_state)                
s_state     = s_tower(n, L) # s^+ (eta^+)^n |0>
s_state = s_state/sparse_norm(s_state)

'''print("⟨P⟩ for eta_tower :", parity_expectation(eta_state, L))
print("⟨P⟩ for t_tower   :", parity_expectation(t_state,   L))
print("⟨P⟩ for s_tower   :", parity_expectation(s_state,   L))'''

# Build the parity operator once
P_op = parity_operator_correct(L)

# Compute expectations using actual operator
print("⟨P⟩ for eta_tower :", parity_of_state_sparse(eta_state, P_op))
print("⟨P⟩ for t_tower   :", parity_of_state_sparse(t_state, P_op))
print("⟨P⟩ for s_tower   :", parity_of_state_sparse(s_state, P_op))


⟨P⟩ for eta_tower : 1.0000000000000187
⟨P⟩ for t_tower   : 1.0000000000000204
⟨P⟩ for s_tower   : 0.9999999999999966


In [ ]:
# now same thing but for subsystems A,B: what's the parity?

# ----------------------------
# 1) Expectation without operator (4^L basis)
# ----------------------------

def _local_counts(label):
    # label in {0,1,2,3} -> (n_up, n_dn, n_tot)
    if label == 0: return (0,0,0)       # |0>
    if label == 1: return (1,0,1)       # |↑>
    if label == 2: return (0,1,1)       # |↓>
    return (1,1,2)                       # |↑↓>

def _digits_base4(idx, L):
    # yield site-local labels in little-endian site order
    for _ in range(L):
        yield (idx & 3)   # idx % 4
        idx >>= 2         # idx //= 4

def parity_expectation_subsystem(state, L, A_sites, atol=1e-12):
    """
    <P_A> for A_sites ⊆ {0,...,L-1}, with P_A = (-1)^{N_A}.
    state: dense vector shape (4**L,) or (4**L,1), or CSR column.
    """
    # flatten to dense 1D
    vec = state.toarray().ravel() if isinstance(state, csr_matrix) else np.asarray(state).ravel()
    nz = np.nonzero(np.abs(vec) > atol)[0]
    if nz.size == 0:
        return 0.0
    A = set(A_sites)
    expP = 0.0
    for i in nz:
        # count N_A by decoding only the needed sites
        Ni_A = 0
        idx = i
        for site in range(L):
            lab = idx & 3
            if site in A:
                Ni_A += _local_counts(lab)[2]
            idx >>= 2
            if idx == 0 and site >= max(A, default=-1):
                # small micro-optimization: stop if higher sites are empty and above max(A)
                pass
        expP += (np.abs(vec[i])**2) * (1.0 if (Ni_A % 2 == 0) else -1.0)
    return float(np.real_if_close(expP))

def analyze_subsystem_parity_simple(state, A, L):
    """
    Simple analysis: compute parity of subsystems A and B directly from reduced states.
    """
    print("=" * 70)
    print("SUBSYSTEM PARITY ANALYSIS (Direct from Reduced States)")
    print("=" * 70)
    
    
    B = [i for i in range(L) if i not in A]
    
    # Parity of subsystem A
    P_A = subsystem_parity_direct(state, L, A)
    
    # Parity of subsystem B (complement)
    P_B = subsystem_parity_direct(state, L, B)
    
    # Product should equal total parity
    P_product = P_A * P_B
    P_total = parity_expectation(state, L)  # Your existing function
    error = abs(P_product - P_total)
    

def parity_expectation_subsystem_spinresolved(state, L, A_sites, which="up", atol=1e-12):
    """
    <(-1)^{N_{A,up}}> if which='up', or <(-1)^{N_{A,down}}> if 'down'.
    """
    vec = state.toarray().ravel() if isinstance(state, csr_matrix) else np.asarray(state).ravel()
    nz = np.nonzero(np.abs(vec) > atol)[0]
    if nz.size == 0:
        return 0.0
    A = set(A_sites)
    expP = 0.0
    for i in nz:
        N_A_sigma = 0
        idx = i
        for site in range(L):
            lab = idx & 3
            if site in A:
                n_up, n_dn, _ = _local_counts(lab)
                N_A_sigma += n_up if which == "up" else n_dn
            idx >>= 2
        expP += (np.abs(vec[i])**2) * (1.0 if (N_A_sigma % 2 == 0) else -1.0)
    return float(np.real_if_close(expP))

# ----------------------------
# 2) Explicit sparse operator P_A (4^L basis)
# ----------------------------

def onsite_parity_matrix():
    # diag(+1, -1, -1, +1) on a single site
    return diags([1, -1, -1, 1], format='csr')

def subsystem_parity_operator(L, A_sites):
    """
    Build P_A as a sparse (4^L x 4^L) operator:
      P_A = ⊗_{i∉A} I_4  ⊗  ⊗_{i∈A} diag(1,-1,-1,1)
    Warning: size grows as 4^L; fine for moderate L.
    """
    I4 = eye(4, format='csr')
    M  = onsite_parity_matrix()
    op = None
    for i in range(L):
        block = M if i in set(A_sites) else I4
        op = block if op is None else kron(op, block, format='csr')
    return op

# ----------------------------
# Example with your towers - including complement B
# ----------------------------

# Choose a subsystem A (e.g., innermost adjacent block)
A = innermost_adjacent_indices(L, 2)  # sites in the middle
B = [i for i in range(L) if i not in A]  # complement of A

print(f"Subsystem A: {A}")
print(f"Subsystem B (complement): {B}")
print()

# Parity expectation values for subsystem A
print("=== Subsystem A Parity ===")
print("<P_A>(eta) =", parity_expectation_subsystem(eta_state, L, A))
print("<P_A>(t)   =", parity_expectation_subsystem(t_state,   L, A))
print("<P_A>(s)   =", parity_expectation_subsystem(s_state,   L, A))
print()

# Parity expectation values for subsystem B (complement)
print("=== Subsystem B (Complement) Parity ===")
print("<P_B>(eta) =", parity_expectation_subsystem(eta_state, L, B))
print("<P_B>(t)   =", parity_expectation_subsystem(t_state,   L, B))
print("<P_B>(s)   =", parity_expectation_subsystem(s_state,   L, B))
print()

# Spin-resolved examples for both subsystems
print("=== Spin-Resolved Parity ===")
print("Subsystem A:")
print("  <P_{A,up}>(eta)   =", parity_expectation_subsystem_spinresolved(eta_state, L, A, which="up"))
print("  <P_{A,down}>(eta) =", parity_expectation_subsystem_spinresolved(eta_state, L, A, which="down"))
print("  <P_{A,up}>(t)     =", parity_expectation_subsystem_spinresolved(t_state, L, A, which="up"))
print("  <P_{A,down}>(t)   =", parity_expectation_subsystem_spinresolved(t_state, L, A, which="down"))
print("  <P_{A,up}>(s)     =", parity_expectation_subsystem_spinresolved(s_state, L, A, which="up"))
print("  <P_{A,down}>(s)   =", parity_expectation_subsystem_spinresolved(s_state, L, A, which="down"))
print()

print("Subsystem B:")
print("  <P_{B,up}>(eta)   =", parity_expectation_subsystem_spinresolved(eta_state, L, B, which="up"))
print("  <P_{B,down}>(eta) =", parity_expectation_subsystem_spinresolved(eta_state, L, B, which="down"))
print("  <P_{B,up}>(t)     =", parity_expectation_subsystem_spinresolved(t_state, L, B, which="up"))
print("  <P_{B,down}>(t)   =", parity_expectation_subsystem_spinresolved(t_state, L, B, which="down"))
print("  <P_{B,up}>(s)     =", parity_expectation_subsystem_spinresolved(s_state, L, B, which="up"))
print("  <P_{B,down}>(s)   =", parity_expectation_subsystem_spinresolved(s_state, L, B, which="down"))
print()

# Verification: P_A * P_B should equal global parity P for total system
print("=== Verification: P_A × P_B = P_total ===")
for state_name, state in [("eta", eta_state), ("t", t_state), ("s", s_state)]:
    P_A_val = parity_expectation_subsystem(state, L, A)
    P_B_val = parity_expectation_subsystem(state, L, B)
    P_total_val = parity_expectation(state, L)
    P_product = P_A_val * P_B_val
    
    print(f"{state_name}: <P_A> × <P_B> = {P_A_val:.6f} × {P_B_val:.6f} = {P_product:.6f}")
    print(f"{state_name}: <P_total> = {P_total_val:.6f}")
    print(f"{state_name}: Difference = {abs(P_product - P_total_val):.2e}")
    print()

# If you want to apply the operators explicitly:
print("=== Explicit operator verification ===")
P_A_op = subsystem_parity_operator(L, A)
P_B_op = subsystem_parity_operator(L, B)

print("⟨eta|P_A|eta⟩ via operator =", (eta_state.T.conj() @ (P_A_op @ eta_state)).toarray().item())
print("⟨eta|P_B|eta⟩ via operator =", (eta_state.T.conj() @ (P_B_op @ eta_state)).toarray().item())

Subsystem A: [5, 6]
Subsystem B (complement): [0, 1, 2, 3, 4, 7, 8, 9, 10, 11]

=== Subsystem A Parity ===
<P_A>(eta) = 1.0000000000000187
<P_A>(t)   = 0.40740740740740455
<P_A>(s)   = 0.6666666666666644

=== Subsystem B (Complement) Parity ===
<P_B>(eta) = 1.0000000000000187
<P_B>(t)   = 0.40740740740740455
<P_B>(s)   = 0.6666666666666644

=== Spin-Resolved Parity ===
Subsystem A:
  <P_{A,up}>(eta)   = -0.09090909090909101
  <P_{A,down}>(eta) = -0.09090909090909101
  <P_{A,up}>(t)     = 0.40740740740740455
  <P_{A,down}>(t)   = 0.11111111111111177
  <P_{A,up}>(s)     = -0.13333333333333297
  <P_{A,down}>(s)   = -0.13333333333333297

Subsystem B:
  <P_{B,up}>(eta)   = -0.09090909090909101
  <P_{B,down}>(eta) = -0.09090909090909101
  <P_{B,up}>(t)     = 0.40740740740740455
  <P_{B,down}>(t)   = 0.11111111111111177
  <P_{B,up}>(s)     = 0.13333333333333297
  <P_{B,down}>(s)   = 0.13333333333333297

=== Verification: P_A × P_B = P_total ===
eta: <P_A> × <P_B> = 1.000000 × 1.000000 = 1.000